In [23]:
# from sqlalchemy import create_engine
import pymysql
import paramiko
import pandas as pd
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
from sqlalchemy import create_engine
from datetime import datetime

In [6]:
tickers = ['ES-21.12', 'ES-21.09', 'ES-21.06', 'ES-21.03', 'ES-20.12', 'ES-20.09', 'ES-20.06', 
           'ES-20.03', 'ES-19.12']

In [10]:
sql_hostname = '23.23.90.225'
sql_username = 'research'
sql_password = '0xResearch'
sql_main_database = 'ib_data'
sql_port = '3306'

engine = create_engine(f'mysql+pymysql://{sql_username}:{sql_password}@{sql_hostname}:{sql_port}/{sql_main_database}')
conn = engine.raw_connection() 
cursor = conn.cursor()
created_tables = set(pd.read_sql_query(f'select name from main_index', conn)['name'])
index = pd.read_sql_query(
            """select main_index.id, 
                main_index.name, 
                main_index.type as type_id, 
                instruments_types.name as type_name 
                from main_index 
                join instruments_types 
                on main_index.type = instruments_types.id;""", conn)
for t in tickers:
    print(t)
    if (index['name'] == t).any() == False:
        query = f"insert into main_index(type, name) values (4, '{t}');"
        cursor.execute(query)
        conn.commit()
        query = f"select id from main_index where name = '{t}';"
        tid = pd.read_sql_query(query, conn)['id'][0]
    else:
        tid = index[index['name'] == t]['id']

    print(f'Starting data transfer to {t}.')

    df = pd.read_csv(t+'.csv', index_col='time')
    df['instrument'] = tid
    df.rename(columns={'date' : 'time'})
    df.to_sql('bars', engine, if_exists='append')

conn.close()

ES-21.12
Starting data transfer to ES-21.12.
ES-21.09
Starting data transfer to ES-21.09.
ES-21.06
Starting data transfer to ES-21.06.
ES-21.03
Starting data transfer to ES-21.03.
ES-20.12
Starting data transfer to ES-20.12.
ES-20.09
Starting data transfer to ES-20.09.
ES-20.06
Starting data transfer to ES-20.06.
ES-20.03
Starting data transfer to ES-20.03.
ES-19.12
Starting data transfer to ES-19.12.


In [58]:
list(data['name'])

['EURCHF']

In [56]:
df = pd.read_csv('1m_data_SPY.csv', index_col='time')

In [57]:
df.head()

,open,high,low,close,volume,instrument,barCount,average
time,,,,,,,,
2004-01-02 13:00:00,111.85,111.85,111.85,111.85,800.0,5,-1,-1
2004-01-02 13:01:00,111.82,111.82,111.82,111.82,200.0,5,-1,-1
2004-01-02 13:03:00,111.83,111.83,111.83,111.83,200.0,5,-1,-1
2004-01-02 13:04:00,111.81,111.81,111.80,111.80,3300.0,5,-1,-1
2004-01-02 13:06:00,111.85,111.85,111.85,111.85,700.0,5,-1,-1


In [55]:
df = df.sort_values('time', axis=0).drop_duplicates(subset=['time'])
df.to_csv('1m_data_SPY.csv')

In [45]:
df.head()

,open,high,low,close,volume,instrument,barCount,average
time,,,,,,,,
2004-01-02 13:00:00,111.85,111.85,111.85,111.85,800.0,5,-1,-1
2004-01-02 13:01:00,111.82,111.82,111.82,111.82,200.0,5,-1,-1
2004-01-02 13:03:00,111.83,111.83,111.83,111.83,200.0,5,-1,-1
2004-01-02 13:04:00,111.81,111.81,111.80,111.80,3300.0,5,-1,-1
2004-01-02 13:06:00,111.85,111.85,111.85,111.85,700.0,5,-1,-1


In [39]:
df = df.rename(columns={'datetime' : 'time'})

In [40]:
df.head()

,time,open,high,low,close,volume,instrument,barCount,average
timestamp,,,,,,,,,
1073048400,2004-01-02 13:00:00,111.85,111.85,111.85,111.85,800.0,3,-1,-1
1073048460,2004-01-02 13:01:00,111.82,111.82,111.82,111.82,200.0,3,-1,-1
1073048580,2004-01-02 13:03:00,111.83,111.83,111.83,111.83,200.0,3,-1,-1
1073048640,2004-01-02 13:04:00,111.81,111.81,111.80,111.80,3300.0,3,-1,-1
1073048760,2004-01-02 13:06:00,111.85,111.85,111.85,111.85,700.0,3,-1,-1


In [59]:
sql_hostname = '23.23.90.225'
sql_username = 'research'
sql_password = '0xResearch'
sql_main_database = 'ib_data'
sql_port = '3306'

engine = create_engine(f'mysql+pymysql://{sql_username}:{sql_password}@{sql_hostname}:{sql_port}/{sql_main_database}')
conn = engine.raw_connection() 
cursor = conn.cursor()
created_tables = set(pd.read_sql_query(f'select name from main_index', conn)['name'])
index = pd.read_sql_query(
            """select main_index.id, 
                main_index.name, 
                main_index.type as type_id, 
                instruments_types.name as type_name 
                from main_index 
                join instruments_types 
                on main_index.type = instruments_types.id;""", conn)
t = 'SPY'
if (index['name'] == t).any() == False:
    query = f"insert into main_index(type, name) values (5, '{t}');"
    cursor.execute(query)
    conn.commit()
    query = f"select id from main_index where name = '{t}';"
    tid = pd.read_sql_query(query, conn)['id'][0]
else:
    tid = index[index['name'] == t]['id'].iloc[0]

print(f'Starting data transfer to {t}.')

df = pd.read_csv('1m_data_SPY.csv', index_col='time')
df['instrument'] = tid
df.rename(columns={'date' : 'time'})
df.to_sql('bars', engine, if_exists='append')

conn.close()

Starting data transfer to SPY.
